In [ ]:

!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  24132      0 --:--:-- --:--:-- --:--:-- 24018
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-nightly ...
     |████████████████████████████████| 61kB 3.5MB/s 
Uninstalling torch-1.7.0+cu101:
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12
Done updating TPU runtime
  Successfully uninstalled torch-1.7.0+cu101
Uninstalling torchvision-0.8.1+cu101:
  Successfully uninstalled torchvision-0.8.1+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly-cp36-cp36m-linux_x86_64.whl...
| [1 files][121.5 MiB/121.5 MiB]                                                
Operation completed over 1 objects/121.5 MiB.                                    
Copying gs://

# Installing Libraries and Downloading Datasets

In [ ]:
! pip install -q datasets transformers tokenizers
from datasets import load_dataset, load_metric

     |████████████████████████████████| 163kB 5.0MB/s 
     |████████████████████████████████| 1.5MB 28.1MB/s 
     |████████████████████████████████| 2.9MB 52.9MB/s 
     |████████████████████████████████| 245kB 52.0MB/s 
     |████████████████████████████████| 17.7MB 240kB/s 
     |████████████████████████████████| 890kB 38.4MB/s 


In [ ]:
!mkdir squad

!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json -O squad/train-v2.0.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json -O squad/dev-v2.0.json

--2020-12-23 15:42:54--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.111.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘squad/train-v2.0.json’

squad/train-v2.0.js 100%[===================>]  40.17M   159MB/s    in 0.3s    

2020-12-23 15:42:54 (159 MB/s) - ‘squad/train-v2.0.json’ saved [42123633/42123633]

--2020-12-23 15:42:54--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.111.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘squad/dev-v2.0.json’

squ

# Reading the datasets

In [ ]:
import json
from pathlib import Path

def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers

train_contexts, train_questions, train_answers = read_squad('squad/train-v2.0.json')
val_contexts, val_questions, val_answers = read_squad('squad/dev-v2.0.json')

# preprocessing the data for QA format

In [ ]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        # sometimes squad answers are off by a character or two – fix this
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters

add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

## Tokenising

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

## Adding Token Position

In [ ]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))
        # if None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)

## Converting the tokens to pytorch data loaders

In [ ]:
import torch

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)


In [ ]:
!pip install -q pytorch_lightning

from torch.utils.data import DataLoader
from transformers import AdamW
import pytorch_lightning as pl
from transformers import DistilBertForQuestionAnswering
import torch_xla.core.xla_model as xm

     |████████████████████████████████| 675kB 5.4MB/s 
     |████████████████████████████████| 102kB 6.7MB/s 
     |████████████████████████████████| 829kB 22.3MB/s 
     |████████████████████████████████| 276kB 35.6MB/s 


# Defining the Model in Pytorch Lightning

In [ ]:
class QA_BERT(pl.LightningModule):
  def __init__(self,lr=0.0005):
    super().__init__()

    self.model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")

    self.lr= lr

  def forward(self,input_ids,attention_mask,start_positions,end_positions):

    return self.model(input_ids,
                      attention_mask=attention_mask,
                      start_positions=start_positions, 
                      end_positions=end_positions
                      )

  def training_step(self, batch, batch_idx):
    input_ids = batch['input_ids'].to(self.device)
    attention_mask = batch['attention_mask'].to(self.device)
    start_positions = batch['start_positions'].to(self.device)
    end_positions = batch['end_positions'].to(self.device)
    outputs = self(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
    loss = outputs[0]
    return loss

  def validation_step(self,batch,batch_idx):
    input_ids = batch['input_ids'].to(self.device)
    attention_mask = batch['attention_mask'].to(self.device)
    start_positions = batch['start_positions'].to(self.device)
    end_positions = batch['end_positions'].to(self.device)
    outputs = self(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
    loss = outputs[0]
    return loss

  def configure_optimizers(self):
    return AdamW(self.parameters(), lr=self.lr)

  def prepare_data(self,stage=None):
    self.train_dataset = SquadDataset(train_encodings)
    self.val_dataset = SquadDataset(val_encodings)

  def setup(self,stage=None):
    train_sampler = torch.utils.data.distributed.DistributedSampler(
            self.train_dataset,
            num_replicas=xm.xrt_world_size(),
            rank=xm.get_ordinal(),
            shuffle=True
        )

    val_sampler = torch.utils.data.distributed.DistributedSampler(
                self.val_dataset,
                num_replicas=xm.xrt_world_size(),
                rank=xm.get_ordinal(),
                shuffle=False
            )


    device  = "cuda" if torch.cuda.is_available() else "cpu"


    self.train_loader = DataLoader(self.train_dataset,
                              batch_size=16,
                              # shuffle= True,
                              sampler=train_sampler,
                              )

    self.val_loader = DataLoader(self.val_dataset, 
                            batch_size=16,
                            # shuffle =False,
                            sampler=val_sampler,
                            )

  def train_dataloader(self):
    return self.train_loader

  def val_dataloader(self):
    return self.val_loader

# Initialising the Model and Training the model

In [ ]:
from pytorch_lightning import Trainer
import tensorflow as tf

trainer = Trainer(max_epochs=3,
                  fast_dev_run=False,
                  gpus=(-1 if torch.cuda.is_available() else 0),
                  auto_lr_find=True,
                  tpu_cores=8,
                  # track_grad_norm = 'inf',
                  # gradient_clip_val =1
                  )

model = QA_BERT()

trainer.tune(model)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: False, used: False
TPU available: True, using: 8 TPU cores


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: cleaning up... please do not interrupt
  warnings.warn(*args, **kwargs)


In [ ]:
trainer.fit(model)